#**Data Cleaning**
Master efficient workflows for cleaning real-world, messy data.

https://www.kaggle.com/learn/data-cleaning

---
**Autor:** Marcos Bezerra

**GitHub:** [https://github.com/marcos-bezerra/Data_Cleaning_Kaggle](https://github.com/marcos-bezerra/Data_Cleaning_Kaggle)

**Google Drive:** [https://drive.google.com/marcos-bezerra/Data_Cleaning_Kaggle](https://drive.google.com/drive/folders/1eHyIT60C7-QV_DaCjMFHBLAaSxbl1ikN?usp=sharing)

**Versão:** 1.0 - 13 Fev 2022

---

## **Lesson 04 - Character Encodings**

Avoid UnicoodeDecodeErrors when loading CSV files.

https://ftfy.readthedocs.io/en/latest/#

https://www.kaggle.com/rtatman/character-encoding-examples

https://www.kaggle.com/alexisbcook/inconsistent-data-entry

In [ ]:
# Autorização para acessar o google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# informa o path da Pasta Raiz da pasta de trabalho
import os
pathRaiz = '/content/drive/MyDrive/I2A2/Desafio_03_Kaggle_DataClean'
os.chdir(pathRaiz)
os.getcwd()

'/content/drive/MyDrive/I2A2/Desafio_03_Kaggle_DataClean'

In [ ]:
ls -lah dataset

total 429M
-rw------- 1 root root 432K Feb 12 12:42  catalog.csv
-rw------- 1 root root  45M Feb 13 12:33  ks-projects-201612.csv
-rw------- 1 root root  47M Feb 13 12:37  ks-projects-201612-utf8.csv
-rw------- 1 root root 263M Sep 20  2019 'NFL Play by Play 2009-2017 (v4).csv'
-rw------- 1 root root  76M Feb  9 11:51 'NFL Play by Play 2009-2017 (v4).csv.zip'
-rw------- 1 root root 231K Feb 12 12:40  pakistan_intellectual_capital.csv


# **Configure nosso ambiente**
A primeira coisa que precisamos fazer é carregar as bibliotecas que usaremos. Mas não é o nosso conjunto de dados: trataremos disso mais tarde!

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful character encoding module
import chardet

# set seed for reproducibility
np.random.seed(0)

**O que são codificações?**

Codificações de caracteres são conjuntos específicos de regras para mapeamento de strings de bytes binários brutos (que se parecem com isso: 0110100001101001) para caracteres que compõem um texto legível por humanos (como "oi"). Existem muitas codificações diferentes e, se você tentar ler um texto com uma codificação diferente daquela em que foi originalmente escrito, acabou com um texto embaralhado chamado "mojibake" (dito como mo-gee-bah-kay). Aqui está um exemplo de mojibake:

æ–‡å—åŒ–ã??

Você também pode acabar com caracteres "desconhecidos". Há o que é impresso quando não há mapeamento entre um byte específico e um caractere na codificação que você está usando para ler sua string de bytes e eles se parecem com isso:

 ���������� 

As incompatibilidades de codificação de caracteres são menos comuns hoje do que costumavam ser, mas definitivamente ainda é um problema. Existem muitas codificações de caracteres diferentes, mas a principal que você precisa saber é UTF-8.

* UTF-8 é a codificação de texto padrão. Todo o código Python está em UTF-8 e, idealmente, todos os seus dados também devem estar. É quando as coisas não estão em UTF-8 que você tem problemas.

Foi muito difícil lidar com codificações no Python 2, mas felizmente no Python 3 é muito mais simples. (Kaggle Notebooks usam apenas Python 3.) Existem dois tipos de dados principais que você encontrará ao trabalhar com texto em Python 3. Um deles é a string, que é o que o texto é por padrão.

In [ ]:
# start with a string
before = "This is the euro symbol: €"

# check to see what datatype it is
type(before)

str

Os outros dados são o tipo de dados bytes, que é uma sequência de inteiros. Você pode converter uma string em bytes especificando em qual codificação ela está:

In [ ]:
# encode it to a different encoding, replacing characters that raise errors
after = before.encode("utf-8", errors="replace")

# check the type
type(after)

bytes

Se você olhar para um objeto bytes, verá que ele tem um b na frente dele, e talvez algum texto depois. Isso porque os bytes são impressos como se fossem caracteres codificados em ASCII. (ASCII é uma codificação de caracteres mais antiga que realmente não funciona para escrever qualquer idioma além do inglês.) Aqui você pode ver que nosso símbolo do euro foi substituído por algum mojibake que se parece com "\xe2\x82\xac" quando é impresso como se fosse uma string ASCII.

In [ ]:
# take a look at what the bytes look like
after

b'This is the euro symbol: \xe2\x82\xac'

Quando convertemos nossos bytes de volta para uma string com a codificação correta, podemos ver que nosso texto está todo lá corretamente, o que é ótimo! :)

In [ ]:
# convert it back to utf-8
print(after.decode("utf-8"))

This is the euro symbol: €


No entanto, quando tentamos usar uma codificação diferente para mapear nossos bytes em uma string, recebemos um erro. Isso ocorre porque a codificação que estamos tentando usar não sabe o que fazer com os bytes que estamos tentando passar. Você precisa informar ao Python a codificação em que a string de bytes deve estar.

Você pode pensar em diferentes codificações como diferentes maneiras de gravar música. Você pode gravar a mesma música em um CD, fita cassete ou 8 faixas. Embora a música possa soar mais ou menos a mesma, você precisa usar o equipamento certo para reproduzir a música de cada formato de gravação. O decodificador correto é como um toca-fitas ou um tocador de CD. Se você tentar reproduzir uma fita em um CD player, simplesmente não funcionará.

In [ ]:
# try to decode our bytes with the ascii encoding
print(after.decode("ascii"))

UnicodeDecodeError: ignored

Também podemos ter problemas se tentarmos usar a codificação errada para mapear de uma string para bytes. Como eu disse anteriormente, as strings são UTF-8 por padrão no Python 3, portanto, se tentarmos tratá-las como se estivessem em outra codificação, criaremos problemas.

Por exemplo, se tentarmos converter uma string em bytes para ASCII usando encode(), podemos pedir que os bytes sejam o que seriam se o texto estivesse em ASCII. Como nosso texto não está em ASCII, porém, haverá alguns caracteres que ele não poderá manipular. Podemos substituir automaticamente os caracteres que o ASCII não pode manipular. Se fizermos isso, no entanto, quaisquer caracteres que não estejam em ASCII serão substituídos pelo caractere desconhecido. Então, quando convertermos os bytes de volta para uma string, o caractere será substituído pelo caractere desconhecido. A parte perigosa disso é que não há como dizer qual personagem deveria ter sido. Isso significa que podemos ter tornado nossos dados inutilizáveis!



In [ ]:
# start with a string
before = "This is the euro symbol: €"

# encode it to a different encoding, replacing characters that raise errors
after = before.encode("ascii", errors = "replace")

# convert it back to utf-8
print(after.decode("ascii"))

# We've lost the original underlying byte string! It's been 
# replaced with the underlying byte string for the unknown character :(

This is the euro symbol: ?


Isso é ruim e queremos evitar fazer isso! É muito melhor converter todo o nosso texto para UTF-8 assim que pudermos e mantê-lo nessa codificação. O melhor momento para converter entradas não UTF-8 em UTF-8 é quando você lê os arquivos, sobre os quais falaremos a seguir.

**Lendo em arquivos com problemas de codificação**

A maioria dos arquivos que você encontrará provavelmente será codificado com UTF-8. Isso é o que o Python espera por padrão, então na maioria das vezes você não terá problemas. No entanto, às vezes você receberá um erro como este:

In [ ]:
# try to read in a file not in UTF-8
kickstarter_2016 = pd.read_csv("dataset/ks-projects-201612.csv")

UnicodeDecodeError: ignored

Observe que obtemos o mesmo **UnicodeDecodeError** que obtivemos quando tentamos decodificar bytes UTF-8 como se fossem ASCII! Isso nos diz que esse arquivo não é realmente UTF-8. No entanto, não sabemos qual codificação é realmente. Uma maneira de descobrir isso é tentar testar várias codificações de caracteres diferentes e ver se alguma delas funciona. Uma maneira melhor, porém, é usar o módulo chardet para tentar adivinhar automaticamente qual é a codificação correta. Não é 100% garantido que esteja certo, mas geralmente é mais rápido do que apenas tentar adivinhar.

Vou apenas olhar para os primeiros dez mil bytes deste arquivo. Isso geralmente é suficiente para um bom palpite sobre qual é a codificação e é muito mais rápido do que tentar examinar o arquivo inteiro. (Especialmente com um arquivo grande, isso pode ser muito lento.) Outra razão para olhar apenas a primeira parte do arquivo é que podemos ver, observando a mensagem de erro, que o primeiro problema é o 11º caractere. Então provavelmente só precisamos olhar para a primeira parte do arquivo para descobrir o que está acontecendo.

In [ ]:
# look at the first ten thousand bytes to guess the character encoding
with open("dataset/ks-projects-201612.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


So chardet is 73% confidence that the right encoding is "Windows-1252". Let's see if that's correct:

In [ ]:
# read in the file with the encoding detected by chardet
kickstarter_2016 = pd.read_csv("dataset/ks-projects-201612.csv", encoding='Windows-1252')

# look at the first few lines
kickstarter_2016.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


Sim, parece que chardet estava certo! O arquivo é lido sem problemas (embora recebamos um aviso sobre os tipos de dados) e quando olhamos para as primeiras linhas, parece estar bem.

E se as suposições do chardet de codificação não estiverem corretas? Como o chardet é basicamente apenas um adivinhador sofisticado, às vezes ele adivinha a codificação errada. Uma coisa que você pode tentar é examinar mais ou menos o arquivo e ver se obtém um resultado diferente e depois tentar isso.

Salvando seus arquivos com codificação UTF-8
Finalmente, uma vez que você passou por todo o trabalho de colocar seu arquivo em UTF-8, você provavelmente vai querer mantê-lo assim. A maneira mais fácil de fazer isso é salvar seus arquivos com codificação UTF-8. A boa notícia é que, como UTF-8 é a codificação padrão em Python, quando você salva um arquivo, ele será salvo como UTF-8 por padrão:

In [ ]:
# save our file (will be saved as UTF-8 by default!)
kickstarter_2016.to_csv("dataset/ks-projects-201612-utf8.csv")